# **Programming for Data Science Final Project**

**Student Information:**

StudentID|Full Name
-|-
21127012|Tran Huy Ban
21127050|Tran Nguyen Huan 
21127143|Nguyen Minh Quan 
21127175|Le Anh Thu


## **Table of contents**

1. [Data Collection](#collect)
   
2. [Data Pre-processing](#process)

    a. [Pre-precessing](#preprocess)

    b. [Exploration](#exploration)
       
3. [Data Modeling](#modeling)

4. [Deploy Model](#deploy)

[References](#references)

## **1. Data Collection** <a name="collect"></a>

## **2. Data Pre-processing** <a name="process"></a>

### **a. Pre-processing** <a name="preprocess"></a>

### **b. Exploration** <a name="exploration"></a>

## **3. Data Modeling** <a name="modeling"></a>

### **4. Deploy Model** <a name="deploy"></a>

## **References** <a name="references"></a>